In [8]:
import pandas as pd
import numpy as np
import torch



def read_file(fname: str, correct_labels=False) -> pd.DataFrame:
    """Reads a filename, return df with text and labels."""

    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    if correct_labels:
        offensive_ids = df.labels != "Acceptable speech"
        df.labels[offensive_ids] = 1
        df.labels[~offensive_ids] = 0
        df["labels"] = df.labels.astype(int)
    df = df.drop(columns=["role"])
    return df

en_test, en_train = "../data/merged-en.test.tsv" , "../data/merged-en.train.tsv"
hr_test, hr_train = "../data/merged-hr.test.tsv" , "../data/merged-hr.train.tsv"
sl_test, sl_train = "../data/merged-sl.test.tsv",  "../data/merged-sl.train.tsv"


In [11]:
def evaluate_HF_model(model_type, model_name, language):
    from simpletransformers.classification import ClassificationModel

    model = ClassificationModel(model_type, model_name)
    
    test_fname = f"../data/merged-{lang}.test.tsv"
    test = read_file(test_fname, correct_labels = True)
    
    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["labels"]
    y_pred = model.predict(list(test["text"].values))[0]

    accuracy = accuracy_score(y_true, y_pred)

    f1 = f1_score(y_true, y_pred, average="macro" )
    
    return accuracy, f1


lang = "hr"
model_type = "electra"
model_name = "./finetuned_models/HR_hate___classla_bcms-bertic_5/"
evaluate_HF_model(model_type, model_name, lang)

Some weights of the model checkpoint at ./finetuned_models/HR_hate___classla_bcms-bertic_5/ were not used when initializing ElectraForSequenceClassification: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at ./finetuned_models/HR_hate___classla_bcms-bertic_5/ and are newly initialized: ['pooler.dense.weight', 'classifier.weight', 'classifi

(0.7882075471698113, 0.7790764090470331)

In [5]:
lang = "hr"
model_type = "electra"
model_name = "classla/bcms-bertic"
evaluate_HF_model(model_type, model_name, lang)

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['pooler.dense.weight', 'classifier.weight

(0.3905660377358491, 0.2808683853459973)

In [28]:
%%time

def train_and_evaluate_model(
    
    model_type: str,
    model_name: str,
    language: str,
                    ):
    from simpletransformers.classification import ClassificationModel
    train_fname = f"../data/merged-{lang}.train.tsv"
    test_fname = f"../data/merged-{lang}.test.tsv"
    train = read_file(train_fname, correct_labels = True)
    test = read_file(test_fname, correct_labels = True)

    
    model_args = {
        "num_train_epochs": 10,
        "learning_rate": 0.00002927,
        "train_batch_size": 80,
        "overwrite_output_dir": True
    }

    model = ClassificationModel(
        model_type, model_name, use_cuda=True,
        args=model_args
    )

    model.train_model(train, )

    #print(model.eval_model(test))
    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["labels"]
    y_pred = model.predict(list(test["text"].values))[0]

    accuracy = accuracy_score(y_true, y_pred)

    f1 = f1_score(y_true, y_pred, average="macro" )
    
    return accuracy, f1


lang = "hr"
model_type = "electra"
model_name = "classla/bcms-bertic"




accs, f1s = list(), list()
for i in range(5):
    a, f = train_and_evaluate_model(model_type, model_name, lang)
    accs.append(a)
    f1s.append(f)
print(f"{accs=}")
print(f"{f1s=}")

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['pooler.dense.weight', 'classifier.weight

KeyboardInterrupt: 

In [18]:
%%time

def train_and_evaluate_model(
    
    model_type: str,
    model_name: str,
    language: str,
                    ):
    from simpletransformers.classification import ClassificationModel
    train_fname = f"../data/merged-{lang}.train.tsv"
    test_fname = f"../data/merged-{lang}.test.tsv"
    train = read_file(train_fname, correct_labels = True)
    test = read_file(test_fname, correct_labels = True)

    
    model_args = {
        "num_train_epochs": 10,
        "learning_rate": 0.00002927,
        "train_batch_size": 40,
        "overwrite_output_dir": True
    }

    model = ClassificationModel(
        model_type, model_name, use_cuda=True,
        args=model_args
    )

    model.train_model(train, )

    #print(model.eval_model(test))
    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["labels"]
    y_pred = model.predict(list(test["text"].values))[0]

    accuracy = accuracy_score(y_true, y_pred)

    f1 = f1_score(y_true, y_pred, average="macro" )

    print(f"""
# Model: {model_name}

|language|accuracy|f1 score|
|---|---|---|
|{lang}|{accuracy:0.3}|{f1:0.3}|

    """)
    
    return accuracy, f1


lang = "hr"
model_type = "electra"
model_name = "classla/bcms-bertic"




accs, f1s = list(), list()
for i in range(5):
    a, f = train_and_evaluate_model(model_type, model_name, lang)
    accs.append(a)
    f1s.append(f)
print(f"{accs=}")
print(f"{f1s=}")

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['pooler.dense.weight', 'classifier.weight



# Model: classla/bcms-bertic

|language|accuracy|f1 score|
|---|---|---|
|hr|0.829|0.819|

    


Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['pooler.dense.weight', 'classifier.weight



# Model: classla/bcms-bertic

|language|accuracy|f1 score|
|---|---|---|
|hr|0.829|0.82|

    


Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['pooler.dense.weight', 'classifier.weight



# Model: classla/bcms-bertic

|language|accuracy|f1 score|
|---|---|---|
|hr|0.829|0.819|

    


Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['pooler.dense.weight', 'classifier.weight



# Model: classla/bcms-bertic

|language|accuracy|f1 score|
|---|---|---|
|hr|0.829|0.819|

    


Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['pooler.dense.weight', 'classifier.weight



# Model: classla/bcms-bertic

|language|accuracy|f1 score|
|---|---|---|
|hr|0.834|0.824|

    
accs=[0.8292452830188679, 0.8287735849056603, 0.8287735849056603, 0.8292452830188679, 0.8344339622641509]
f1s=[0.8193571955830735, 0.8200411522922444, 0.8186453830997273, 0.8191022243090933, 0.8240507900477043]
CPU times: user 18min 59s, sys: 6min 52s, total: 25min 51s
Wall time: 26min 36s


In [22]:
from scipy import stats
a = [
    
    0.829,
0.836,
0.832,
0.832,
0.835,
0.833,
0.837,
]
b = [0.8292452830188679, 0.8287735849056603, 0.8287735849056603, 0.8292452830188679, 0.8344339622641509]
tStat, pValue = stats.ttest_ind(a, b, equal_var = True)
pValue

0.05607924279825321

In [26]:
tStat, pValue = stats.wilcoxon(a[0:len(b)],b,alternative="greater")
pValue

0.0625

In [27]:
lang = "hr"
model_type = "bert"
model_name = "EMBEDDIA/crosloengual-bert"




accs, f1s = list(), list()
for i in range(5):
    a, f = train_and_evaluate_model(model_type, model_name, lang)
    accs.append(a)
    f1s.append(f)
print(f"{accs=}")
print(f"{f1s=}")

Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(




# Model: EMBEDDIA/crosloengual-bert

|language|accuracy|f1 score|
|---|---|---|
|hr|0.81|0.8|

    


Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec



# Model: EMBEDDIA/crosloengual-bert

|language|accuracy|f1 score|
|---|---|---|
|hr|0.803|0.792|

    


...



# Model: EMBEDDIA/crosloengual-bert

|language|accuracy|f1 score|
|---|---|---|
|hr|0.8|0.791|

    


Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec



# Model: EMBEDDIA/crosloengual-bert

|language|accuracy|f1 score|
|---|---|---|
|hr|0.808|0.799|

    


Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec



# Model: EMBEDDIA/crosloengual-bert

|language|accuracy|f1 score|
|---|---|---|
|hr|0.805|0.795|

    
accs=[0.810377358490566, 0.8028301886792453, 0.8, 0.8075471698113208, 0.8047169811320755]
f1s=[0.7996756223115435, 0.7924549689862858, 0.7909970238095237, 0.7990476509094153, 0.7947127925122843]


In [29]:
lang = "sl"
model_type = "bert"
model_name = "EMBEDDIA/crosloengual-bert"




accs, f1s = list(), list()
for i in range(5):
    a, f = train_and_evaluate_model(model_type, model_name, lang)
    accs.append(a)
    f1s.append(f)
print("""
|language|model|accuracy|f1 score|
|---|---|---|---|""")
for ac, f1 in zip(accs, f1s):
    print(f"|{lang}|{model_name}|{ac}|{f1}|")


Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec

Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec

Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec

Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec

Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec



|language|model|accuracy|f1 score|
|---|---|---|---|
|sl|EMBEDDIA/crosloengual-bert|0.7566574839302111|0.75207836311669|
|sl|EMBEDDIA/crosloengual-bert|0.7557392102846648|0.7525306587956893|
|sl|EMBEDDIA/crosloengual-bert|0.7658402203856749|0.7605410816977487|
|sl|EMBEDDIA/crosloengual-bert|0.7575757575757576|0.75350182325449|
|sl|EMBEDDIA/crosloengual-bert|0.7617079889807162|0.7574971869526088|


# Repeated training

In [4]:
%%time


def train_repeatedly(
    
    model_type: str,
    model_name: str,
    language: str,
    num_iter: int,
                    ):
    from simpletransformers.classification import ClassificationModel
    from sklearn.metrics import accuracy_score, f1_score
    
    train_fname = f"../data/merged-{lang}.train.tsv"
    test_fname = f"../data/merged-{lang}.test.tsv"
    train = read_file(train_fname, correct_labels = True)
    test = read_file(test_fname, correct_labels = True)

    
    model_args = {
        "num_train_epochs": 10,
        "learning_rate": 0.00002927,
        "train_batch_size": 80,
        "overwrite_output_dir": True
    }

    model = ClassificationModel(
        model_type, model_name, use_cuda=True,
        args=model_args
    )

    
    
    accs, f1s = list(), list()
    for i in range(num_iter):
        model.train_model(train, )
        y_true = test["labels"]
        y_pred = model.predict(list(test["text"].values))[0]
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average="macro" )
        accs.append(accuracy)
        f1s.append(f1)
    print("""
|language|model|accuracy|f1 score|
|---|---|---|---|""")
    for ac, f1 in zip(accs, f1s):
        print(f"|{lang}|{model_name}|{ac}|{f1}|")


lang = "hr"
model_type = "electra"
model_name = "classla/bcms-bertic"

train_repeatedly(model_type, model_name, lang, 7)

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.bias', 'pooler.dense.weight',

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(




|language|model|accuracy|f1 score|
|---|---|---|---|
|hr|classla/bcms-bertic|0.8306603773584905|0.8217099753442147|
|hr|classla/bcms-bertic|0.8297169811320755|0.8199819304498734|
|hr|classla/bcms-bertic|0.8179245283018868|0.8082582599469894|
|hr|classla/bcms-bertic|0.8221698113207547|0.8128537646660208|
|hr|classla/bcms-bertic|0.8287735849056603|0.8189004265333137|
|hr|classla/bcms-bertic|0.8231132075471698|0.8135133176121541|
|hr|classla/bcms-bertic|0.8306603773584905|0.8201279958480168|
CPU times: user 19min 36s, sys: 9min 57s, total: 29min 34s
Wall time: 30min 12s


In [8]:



def train_and_return_model(
    
    model_type: str,
    model_name: str,
    language: str,
                    ):
    from simpletransformers.classification import ClassificationModel
    from sklearn.metrics import accuracy_score, f1_score
    
    train_fname = f"../data/merged-{lang}.train.tsv"
    test_fname = f"../data/merged-{lang}.test.tsv"
    train = read_file(train_fname, correct_labels = True)
    test = read_file(test_fname, correct_labels = True)

    
    model_args = {
        "num_train_epochs": 10,
        "learning_rate": 0.00002927,
        "train_batch_size": 80,
        "overwrite_output_dir": True
    }

    model = ClassificationModel(
        model_type, model_name, use_cuda=True,
        args=model_args
    )

    
    

    model.train_model(train, )
    y_true = test["labels"]
    y_pred = model.predict(list(test["text"].values))[0]
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="macro" )
    print("""
|language|model|accuracy|f1 score|
|---|---|---|""")
    print(f"|{lang}||{accuracy:0.3}|{f1:0.3}|")
    return model

lang = "hr"
model_type = "electra"
model_name = "classla/bcms-bertic"

model = train_and_return_model(model_type, model_name, lang)

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.bias', 'pooler.dense.weight',

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(




|language|model|accuracy|f1 score|
|---|---|---|
|hr||0.828|0.816|


In [9]:
!mkdir test_model

In [10]:
model.save_model(output_dir="./test_model/", )

In [11]:
!pwd

/home/peterr/macocu/task1/task2


In [17]:
!ls test_model/*

ls: cannot access 'test_model/*': No such file or directory
